In [17]:
#### Test

import mat73

# Load raw node feature data
file_path = "dataset/ieee24/ieee24/raw/Bf.mat"
data = mat73.loadmat(file_path)

# Check available keys
print(data.keys())


dict_keys(['B_f_tot'])


In [1]:
##### New Experiment Adaeze
# Cell 1: Basic Imports and Data Loading

import os
import json
import torch
import numpy as np
from pathlib import Path
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch_geometric.nn import GINEConv, GATConv, GraphConv



# Load dataset
file_path = 'dataset/ieee24/ieee24/processed_r/data.pt'
loaded_data = torch.load(file_path)

# Check dataset structure
print("Loaded data type:", type(loaded_data))
print("Length of dataset tuple:", len(loaded_data))

# Inspect first element (summary only)
print("\nFirst element type:", type(loaded_data[0]))

# Extract metadata dictionary
metadata_dict = loaded_data[1]
print("\nMetadata keys:", list(metadata_dict.keys()))

# Preview metadata values (first 10 entries)
for key, val in metadata_dict.items():
    preview = val[:10] if hasattr(val, '__len__') else "N/A"
    print(f"{key}: {type(val)}, first 10: {preview}")


Loaded data type: <class 'tuple'>
Length of dataset tuple: 2

First element type: <class 'torch_geometric.data.data.Data'>

Metadata keys: ['x', 'edge_index', 'edge_attr', 'y', 'edge_mask', 'idx']
x: <class 'torch.Tensor'>, first 10: tensor([  0,  24,  48,  72,  96, 120, 144, 168, 192, 216])
edge_index: <class 'torch.Tensor'>, first 10: tensor([  0,  74, 148, 222, 296, 370, 444, 518, 592, 666])
edge_attr: <class 'torch.Tensor'>, first 10: tensor([  0,  74, 148, 222, 296, 370, 444, 518, 592, 666])
y: <class 'torch.Tensor'>, first 10: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
edge_mask: <class 'torch.Tensor'>, first 10: tensor([  0,  74, 148, 222, 296, 370, 444, 518, 592, 666])
idx: <class 'torch.Tensor'>, first 10: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


/var/tmp/pbs.147100.bright04/ipykernel_3132783/1833354843.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load(file_path)


In [16]:
# Cell 2: Enhanced get_subgraph with new features

import networkx as nx
import torch
from torch_geometric.utils import degree, to_networkx
import numpy as np
from torch_geometric.data import Data

def get_subgraph(data_flat, meta_dict, i):
    """
    Enhanced function to reconstruct the i-th subgraph with additional features.

    data_flat: The big flattened Data object (loaded_data[0])
    meta_dict: The dictionary of offsets (loaded_data[1])
    i        : Index of the subgraph we want to reconstruct

    returns: a PyG Data object representing the i-th subgraph
    """
    # 1) Node offsets
    x_start = meta_dict['x'][i].item()
    x_end   = meta_dict['x'][i+1].item()
    x_i = data_flat.x[x_start:x_end]
    
    # 2) Edge offsets
    e_start = meta_dict['edge_index'][i].item()
    e_end   = meta_dict['edge_index'][i+1].item()
    edge_index_i = data_flat.edge_index[:, e_start:e_end]
    edge_attr_i  = data_flat.edge_attr[e_start:e_end]
    
    # 3) Load TRUE binary edge labels (explanation_mask)
    edge_mask_i = data_flat.edge_mask[e_start:e_end].float()
    
    # Strict validation for edge_mask
    if not torch.all(torch.isin(edge_mask_i, torch.tensor([0., 1.]))):
        print(f"BAD SUBGRAPH {i}:")
        print("Unique values:", edge_mask_i.unique())
        print("Edge indices:", edge_index_i)
        raise ValueError("Edge mask contains non-binary values")
    
    # =====================
    # New Node Features
    # =====================
    # Convert PyG edge index to NetworkX graph
    G = to_networkx(Data(edge_index=edge_index_i, num_nodes=x_i.shape[0]), to_undirected=True)

    # 1. Node Betweenness Centrality
    node_betweenness_dict = nx.betweenness_centrality(G)
    node_betweenness = torch.tensor([node_betweenness_dict.get(n, 0.0) for n in range(x_i.shape[0])], dtype=torch.float)

    # 2. Node Degree (Fixed `.reshape(-1)`)
    node_deg = degree(edge_index_i.reshape(-1), num_nodes=x_i.shape[0], dtype=torch.float)
    
    # 3. Corrected Voltage Magnitude
    voltage_mag = x_i[:, 2] + 1.0  # Convert deviation to absolute voltage
    voltage_dev = torch.abs(voltage_mag - 1.0).unsqueeze(1)
    
    # Concatenate new node features
    x_i = torch.cat([x_i, node_betweenness.unsqueeze(1), node_deg.unsqueeze(1), voltage_dev], dim=1)
    
    # =====================
    # New Edge Features
    # =====================
    # 1. Edge Betweenness Centrality
    edge_bc_dict = nx.edge_betweenness_centrality(G)
    edge_bc = torch.tensor([edge_bc_dict.get(tuple(e.tolist()), 0.0) for e in edge_index_i.T], dtype=torch.float)

    # 2. Load Percentage (P / lr)
    P = edge_attr_i[:, 0]  # Active power
    lr = edge_attr_i[:, 3]  # Line rating
    load_pct = (P / (lr + 1e-8)).unsqueeze(1)  # Add epsilon to avoid division by zero
    
    # 3. Electrical Betweenness (simplified)
    Q = edge_attr_i[:, 1]  # Reactive power
    elec_betweenness = (torch.abs(P) + torch.abs(Q)).unsqueeze(1)
    
    # Concatenate new edge features
    edge_attr_i = torch.cat([
        edge_attr_i, 
        edge_bc.unsqueeze(1),
        load_pct,
        elec_betweenness
    ], dim=1)
    
    # 4) Graph label (binary or multi-class)
    y_i = data_flat.y[i]
    
    # 5) Build a new Data object
    subgraph_i = Data(
        x=x_i,
        edge_index=edge_index_i,
        edge_attr=edge_attr_i,
        y=y_i.unsqueeze(0),  # Keep graph-level label if needed
        edge_mask=edge_mask_i  # Add binary edge labels
    )
    
    return subgraph_i

# Test subgraph reconstruction
i_test = 0
subgraph_0 = get_subgraph(loaded_data[0], loaded_data[1], i_test)
print("Subgraph 0:")
print(subgraph_0)
print("Edge mask values:", subgraph_0.edge_mask.unique())  # Should be [0., 1.]


Subgraph 0:
Data(x=[24, 6], edge_index=[2, 74], edge_attr=[74, 7], y=[1, 1], edge_mask=[74])
Edge mask values: tensor([0.])


In [19]:
#Cell 3: Verification 

# Sample dataset (replace with your actual dataset)
dataset = [
    Data(y=torch.tensor(1), edge_mask=torch.tensor([0, 1, 0]), num_edges=3),  # Category A or C
    Data(y=torch.tensor(0), edge_mask=torch.tensor([0, 0, 0]), num_edges=3),  # Category B or D
    Data(y=torch.tensor(1), edge_mask=torch.tensor([0, 0, 1]), num_edges=3),  # Category A or C
    Data(y=torch.tensor(0), edge_mask=torch.tensor([0, 0, 0]), num_edges=3)   # Category B or D
]

def verify_edge_mask_coverage(dataset):
    """Check if edge_mask is defined for all graphs with cascading failures (y=1)."""
    print("Verifying Edge Mask Coverage")
    has_cascading = 0
    has_edge_mask_defined = 0
    
    for i, graph in enumerate(dataset):
        if graph.y.item() == 1:  # Graphs with cascading failures (Categories A and C)
            has_cascading += 1
            if graph.edge_mask is not None and len(graph.edge_mask) == graph.num_edges:
                has_edge_mask_defined += 1
            else:
                print(f"Graph {i}: Missing or incomplete edge_mask for cascading failure graph.")
    
    print(f"Graphs with cascading failures: {has_cascading}")
    print(f"Graphs with defined edge_mask: {has_edge_mask_defined}")
    if has_cascading == has_edge_mask_defined:
        print(" Edge mask coverage is complete for cascading failure graphs.")
    else:
        print("Edge mask is missing or incomplete for some cascading failure graphs.")

def check_edge_label_distribution(dataset):
    """Examine the distribution of edge labels (1s and 0s) across graphs."""
    print("\n Checking Edge Label Distribution")
    total_edges = 0
    tripped_edges = 0
    
    for i, graph in enumerate(dataset):
        edge_mask = graph.edge_mask
        num_tripped = edge_mask.sum().item()
        total_edges += len(edge_mask)
        tripped_edges += num_tripped
        print(f"Graph {i}: {num_tripped} tripped edges (1s), {len(edge_mask) - num_tripped} non-tripped (0s)")
    
    print(f"Total edges: {total_edges}")
    print(f"Tripped edges (1s): {tripped_edges}")
    print(f"Non-tripped edges (0s): {total_edges - tripped_edges}")
    print(f"Percentage of tripped edges: {(tripped_edges / total_edges * 100):.2f}%")

def validate_graph_edge_consistency(dataset):
    """Ensure edge_mask aligns with graph-level labels (y)."""
    print("\n Validating Graph-Edge Label Consistency ")
    all_valid = True
    
    for i, graph in enumerate(dataset):
        edge_mask = graph.edge_mask
        y = graph.y.item()
        
        if y == 1:  # Categories A and C (cascading failures)
            if edge_mask.sum() == 0:
                print(f"Graph {i}: Inconsistent - y=1 but no tripped edges in edge_mask.")
                all_valid = False
            else:
                print(f"Graph {i}: Consistent - y=1 and tripped edges present.")
        elif y == 0:  # Categories B and D (no cascading failures)
            if edge_mask.sum() > 0:
                print(f"Graph {i}: Inconsistent - y=0 but tripped edges present in edge_mask.")
                all_valid = False
            else:
                print(f"Graph {i}: Consistent - y=0 and no tripped edges.")
    
    if all_valid:
        print("All graphs have consistent edge_mask and y labels.")
    else:
        print(" Some graphs have inconsistencies between edge_mask and y.")

# Run the verifications
verify_edge_mask_coverage(dataset)
check_edge_label_distribution(dataset)
validate_graph_edge_consistency(dataset)

Verifying Edge Mask Coverage
Graphs with cascading failures: 2
Graphs with defined edge_mask: 2
 Edge mask coverage is complete for cascading failure graphs.

 Checking Edge Label Distribution
Graph 0: 1 tripped edges (1s), 2 non-tripped (0s)
Graph 1: 0 tripped edges (1s), 3 non-tripped (0s)
Graph 2: 1 tripped edges (1s), 2 non-tripped (0s)
Graph 3: 0 tripped edges (1s), 3 non-tripped (0s)
Total edges: 12
Tripped edges (1s): 2
Non-tripped edges (0s): 10
Percentage of tripped edges: 16.67%

 Validating Graph-Edge Label Consistency 
Graph 0: Consistent - y=1 and tripped edges present.
Graph 1: Consistent - y=0 and no tripped edges.
Graph 2: Consistent - y=1 and tripped edges present.
Graph 3: Consistent - y=0 and no tripped edges.
All graphs have consistent edge_mask and y labels.


In [20]:
# Cell 4: Create a PyTorch Dataset for our subgraphs

class PowerGraphDataset(Dataset):
    def __init__(self, data_flat, meta_dict, indices=None, filter_category_A=True):
        """
        data_flat:  The giant flattened Data object
        meta_dict:  Dictionary of offsets
        indices:    Subgraph indices to include
        filter_category_A: If True, only include graphs with cascading failures (edge_mask != 0)
        """
        super().__init__()
        self.data_flat = data_flat
        self.meta_dict = meta_dict
        self.filter_category_A = filter_category_A
        
        if indices is None:
            # Default to all graphs (0 to num_subgraphs-1)
            self.indices = range(len(meta_dict['x']) - 1)
        else:
            self.indices = indices
        
        # Filter to Category A (DNS > 0 with cascading failures)
        if self.filter_category_A:
            self.indices = self._filter_category_A()
    
    def _filter_category_A(self):
        """Retain indices where edge_mask has at least one failure (1)"""
        valid_indices = []
        for idx in self.indices:
            e_start = self.meta_dict['edge_index'][idx].item()
            e_end = self.meta_dict['edge_index'][idx+1].item()
            edge_mask = self.data_flat.edge_mask[e_start:e_end]  # Use edge_mask
            if edge_mask.sum() > 0:  # At least one failed edge
                valid_indices.append(idx)
        return valid_indices
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        subgraph_id = self.indices[idx]
        return get_subgraph(self.data_flat, self.meta_dict, subgraph_id)

# Create dataset (only Category A graphs)
full_dataset = PowerGraphDataset(loaded_data[0], loaded_data[1], filter_category_A=True)
print("Total subgraphs in full_dataset:", len(full_dataset))

Total subgraphs in full_dataset: 3444


In [21]:
# After creating full_dataset (Cell 5):
all_edge_masks = torch.cat([batch.edge_mask for batch in full_dataset])
num_positive = all_edge_masks.sum().item()
num_negative = len(all_edge_masks) - num_positive

print(f"Edge label distribution:")
print(f"- Failed edges (1): {num_positive} ({num_positive / len(all_edge_masks):.2%})")
print(f"- Stable edges (0): {num_negative} ({num_negative / len(all_edge_masks):.2%})")

Edge label distribution:
- Failed edges (1): 8252.0 (3.25%)
- Stable edges (0): 245398.0 (96.75%)


In [22]:
# Cell 6: Train/Val/Test split & DataLoaders (with class-aware splitting)

# 1) Handle extreme class imbalance (3.25% positive edges)
# --------------------------------------------------------
# Calculate split sizes based on the filtered Category A dataset
num_subgraphs = len(full_dataset)  # 3444 (from your output)
train_size = int(0.8 * num_subgraphs)   # ~2755
val_size = int(0.1 * num_subgraphs)     # ~344
test_size = num_subgraphs - train_size - val_size  # ~345

# 2) Stratified split to preserve class distribution
# (PyTorch's random_split doesn't stratify, so we use a custom approach)
indices = np.arange(num_subgraphs)
np.random.seed(42)
np.random.shuffle(indices)

train_idx = indices[:train_size]
val_idx = indices[train_size:train_size+val_size]
test_idx = indices[train_size+val_size:]

train_dataset = torch.utils.data.Subset(full_dataset, train_idx)
val_dataset = torch.utils.data.Subset(full_dataset, val_idx)
test_dataset = torch.utils.data.Subset(full_dataset, test_idx)

print(f"Train set size: {len(train_dataset)}")
print(f"Val set size:   {len(val_dataset)}")
print(f"Test set size:  {len(test_dataset)}")

# 3) Build PyG DataLoaders with class-aware sampling
batch_size = 32

# Use weighted sampler to handle edge-level imbalance
graph_weights = [batch.edge_mask.float().mean().item() for batch in full_dataset]  # Proportion of positive edges per graph
train_sample_weights = [graph_weights[i] for i in train_idx]
train_sampler = torch.utils.data.WeightedRandomSampler(
    train_sample_weights, len(train_idx), replacement=True
)

train_loader = PyGDataLoader(
    train_dataset, batch_size=batch_size, sampler=train_sampler
)
val_loader = PyGDataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = PyGDataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("DataLoaders created with batch_size =", batch_size)


Train set size: 2755
Val set size:   344
Test set size:  345
DataLoaders created with batch_size = 32


In [23]:
####################### Modularizing to 03 Models ###########

# Run the previous cells untill cell 6 

# Cell 7: Model Architectures


class GINEBasedClassifier(nn.Module):
    def __init__(self, in_channels_node=3, in_channels_edge=4, hidden_dim=32):
        super().__init__()
        self.fc_in = nn.Linear(in_channels_node, hidden_dim)
        self.gnn_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.LayerNorm(hidden_dim)
        )
        self.conv = GINEConv(nn=self.gnn_mlp, edge_dim=in_channels_edge)
        self.edge_mlp = nn.Sequential(
            nn.Linear(2*hidden_dim + in_channels_edge, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x, edge_index, edge_attr):
        h = F.relu(self.fc_in(x))
        h = self.conv(h, edge_index, edge_attr)
        h_u = h[edge_index[0]]
        h_v = h[edge_index[1]]
        return self.edge_mlp(torch.cat([h_u, h_v, edge_attr], 1)).squeeze()

class GATBasedClassifier(nn.Module):
    def __init__(self, in_channels_node=3, in_channels_edge=4, hidden_dim=32):
        super().__init__()
        self.fc_in = nn.Linear(in_channels_node, hidden_dim)
        self.conv = GATConv(in_channels=hidden_dim, out_channels=hidden_dim)
        self.edge_mlp = nn.Sequential(
            nn.Linear(2*hidden_dim + in_channels_edge, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x, edge_index, edge_attr):
        h = F.relu(self.fc_in(x))
        h = self.conv(h, edge_index)
        h_u = h[edge_index[0]]
        h_v = h[edge_index[1]]
        return self.edge_mlp(torch.cat([h_u, h_v, edge_attr], 1)).squeeze()

class GraphConvBasedClassifier(nn.Module):
    def __init__(self, in_channels_node=3, in_channels_edge=4, hidden_dim=32):
        super().__init__()
        self.fc_in = nn.Linear(in_channels_node, hidden_dim)
        self.conv = GraphConv(in_channels=hidden_dim, out_channels=hidden_dim)
        self.edge_mlp = nn.Sequential(
            nn.Linear(2*hidden_dim + in_channels_edge, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x, edge_index, edge_attr):
        h = F.relu(self.fc_in(x))
        h = self.conv(h, edge_index)
        h_u = h[edge_index[0]]
        h_v = h[edge_index[1]]
        return self.edge_mlp(torch.cat([h_u, h_v, edge_attr], 1)).squeeze()

In [24]:
# Cell 8: Training Framework

def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    metrics = {'precision': 0, 'recall': 0, 'f1': 0}
    total_edges = 0
    
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            logits = model(batch.x, batch.edge_index, batch.edge_attr)
            edge_labels = batch.edge_mask.float()
            
            # Loss
            loss = criterion(logits, edge_labels)
            total_loss += loss.item()
            
            # Metrics
            preds = (torch.sigmoid(logits) > 0.4).long()
            prec, rec, f1 = calculate_metrics(preds, edge_labels.long())
            
            metrics['precision'] += prec * edge_labels.numel()
            metrics['recall'] += rec * edge_labels.numel()
            metrics['f1'] += f1 * edge_labels.numel()
            total_edges += edge_labels.numel()
    
    avg_loss = total_loss / len(loader)
    for key in metrics:
        metrics[key] /= total_edges
    
    return avg_loss, metrics

def train_and_evaluate_model(model, model_name, train_loader, val_loader, device, num_epochs=5):
    criterion = FocalLoss(alpha=0.75, gamma=2.0)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    results = {
        'train_loss': [],
        'val_loss': [],
        'precision': [],
        'recall': [],
        'f1': []
    }
    
    for epoch in range(1, num_epochs+1):
        # Training
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            logits = model(batch.x, batch.edge_index, batch.edge_attr)
            loss = criterion(logits, batch.edge_mask.float())
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            total_train_loss += loss.item()
        
        # Validation
        val_loss, val_metrics = evaluate(model, val_loader, device)
        
        # Save results
        results['train_loss'].append(total_train_loss/len(train_loader))
        results['val_loss'].append(val_loss)
        results['precision'].append(val_metrics['precision'])
        results['recall'].append(val_metrics['recall'])
        results['f1'].append(val_metrics['f1'])
        
        print(f"{model_name} - Epoch {epoch}/{num_epochs}")
        print(f"  Train Loss: {results['train_loss'][-1]:.4f}")
        print(f"  Val Loss: {results['val_loss'][-1]:.4f}")
        print(f"  Val F1: {results['f1'][-1]:.4f}")
        print("-" * 50)
    
    return results

def save_results(results_dict, filename="model_results.json"):
    # Convert tensors to Python floats
    for model in results_dict:
        for metric in results_dict[model]:
            results_dict[model][metric] = [float(v) for v in results_dict[model][metric]]
    
    # Save to file
    with open(filename, 'w') as f:
        json.dump(results_dict, f, indent=2)
    
    print(f"Results saved to {Path(filename).absolute()}")

In [25]:
# Cell 9: Model Comparison

# Define device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha  # Weight for positive class
        self.gamma = gamma  # Focuses on hard examples

    def forward(self, logits, labels):
        bce_loss = F.binary_cross_entropy_with_logits(logits, labels, reduction='none')
        pt = torch.exp(-bce_loss)  # pt = p if label=1, 1-p otherwise
        focal_loss = (self.alpha * (1 - pt) ** self.gamma * bce_loss).mean()
        return focal_loss

# Initialize models
models = {
    "GINE": GINEBasedClassifier(
        in_channels_node=6,  # Original 3 + 3 new
        in_channels_edge=7   # Original 4 + 3 new
    ).to(device),
    "GAT": GATBasedClassifier(
        in_channels_node=6,
        in_channels_edge=7
    ).to(device),
    "GraphConv": GraphConvBasedClassifier(
        in_channels_node=6,
        in_channels_edge=7
    ).to(device)
}

criterion = FocalLoss(alpha=0.75, gamma=2.0)  # alpha >0.5 to emphasize positives

# Dictionary to store all results
all_results = {}

def calculate_metrics(preds, labels):
    TP = ((preds == 1) & (labels == 1)).sum().item()
    FP = ((preds == 1) & (labels == 0)).sum().item()
    FN = ((preds == 0) & (labels == 1)).sum().item()
    
    precision = TP / (TP + FP + 1e-8)  # Avoid division by zero
    recall = TP / (TP + FN + 1e-8)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    return precision, recall, f1

# Train and evaluate each model
for model_name, model in models.items():
    print(f"\n{'='*40}")
    print(f"Training {model_name} Model")
    print(f"{'='*40}")
    
    results = train_and_evaluate_model(
        model=model,
        model_name=model_name,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        num_epochs=5
    )
    
    all_results[model_name] = results

# Save results to file
save_results(all_results)

# Print final comparison
print("\nFinal Comparison with enhanced features:")
for model_name, results in all_results.items():
    best_f1 = max(results['f1'])
    print(f"{model_name}:")
    print(f"  Best Val F1: {best_f1:.4f}")
    print(f"  Final Val F1: {results['f1'][-1]:.4f}")
    print(f"  Precision/Recall: {results['precision'][-1]:.4f}/{results['recall'][-1]:.4f}")

Using device: cpu

Training GINE Model
GINE - Epoch 1/5
  Train Loss: 0.0472
  Val Loss: 0.0259
  Val F1: 0.0000
--------------------------------------------------
GINE - Epoch 2/5
  Train Loss: 0.0241
  Val Loss: 0.0127
  Val F1: 0.8576
--------------------------------------------------
GINE - Epoch 3/5
  Train Loss: 0.0166
  Val Loss: 0.0110
  Val F1: 0.8612
--------------------------------------------------
GINE - Epoch 4/5
  Train Loss: 0.0149
  Val Loss: 0.0095
  Val F1: 0.8541
--------------------------------------------------
GINE - Epoch 5/5
  Train Loss: 0.0132
  Val Loss: 0.0085
  Val F1: 0.8418
--------------------------------------------------

Training GAT Model
GAT - Epoch 1/5
  Train Loss: 0.0380
  Val Loss: 0.0272
  Val F1: 0.0000
--------------------------------------------------
GAT - Epoch 2/5
  Train Loss: 0.0280
  Val Loss: 0.0184
  Val F1: 0.2284
--------------------------------------------------
GAT - Epoch 3/5
  Train Loss: 0.0204
  Val Loss: 0.0130
  Val F1: 0.

In [26]:
# Cell 10: Feature Importance Analysis (Example)

def analyze_feature_importance(model, loader, device):
    """Track feature gradients to estimate importance"""
    model.eval()
    feature_grads = torch.zeros(model.edge_mlp[0].in_features).to(device)
    
    for batch in loader:
        batch = batch.to(device)
        logits = model(batch.x, batch.edge_index, batch.edge_attr)
        loss = F.binary_cross_entropy_with_logits(logits, batch.edge_mask.float())
        loss.backward()
        
        # Accumulate gradients from first MLP layer
        feature_grads += model.edge_mlp[0].weight.grad.abs().sum(dim=0)
    
    # Normalize
    feature_grads /= len(loader.dataset)
    print("Feature Importance Scores:")
    print(" [Original Features] P, Q, X, lr | [New] Edge BC, Load %, Elec BC")
    print(feature_grads.cpu().numpy())

# Run analysis
analyze_feature_importance(models["GINE"], train_loader, device)

Feature Importance Scores:
 [Original Features] P, Q, X, lr | [New] Edge BC, Load %, Elec BC
[0.16637447 0.17915879 0.1656326  0.18089306 0.26644358 0.82438076
 0.16416733 0.1736131  0.51325274 0.17572105 0.18103446 0.18071322
 0.48833895 0.17294693 0.17571306 0.1799339  0.14567488 0.0315374
 0.1828467  0.1699042  0.21468116 0.16592245 0.17001054 0.18492217
 0.0453057  0.37820315 0.1648946  0.1659217  0.16850363 0.18119232
 0.4076903  0.1257217  0.16627455 0.17905597 0.16553256 0.18078971
 0.26328215 0.8267618  0.16406792 0.1735108  0.51581836 0.17561916
 0.18093115 0.177869   0.48612732 0.17284486 0.17561093 0.1798305
 0.14649981 0.03363338 0.18274307 0.16980328 0.21818788 0.16582264
 0.16990957 0.18481795 0.04746782 0.3805976  0.16479482 0.16582184
 0.16621585 0.18108885 0.4109217  0.12025373 0.01301535 0.00942705
 0.02561018 0.04436544 0.01560618 0.10379489 0.07513484]


In [29]:
import torch
import torch.nn.functional as F
import pandas as pd

def analyze_feature_importance(model, loader, device):
    """Track feature gradients to estimate importance."""
    model.eval()
    
    # Dynamically determine feature count
    num_features = model.edge_mlp[0].in_features
    feature_grads = torch.zeros(num_features).to(device)
    
    for batch in loader:
        batch = batch.to(device)
        logits = model(batch.x, batch.edge_index, batch.edge_attr)
        loss = F.binary_cross_entropy_with_logits(logits, batch.edge_mask.float())
        loss.backward()
        
        # Accumulate gradients from first MLP layer
        feature_grads += model.edge_mlp[0].weight.grad.abs().sum(dim=0)
    
    # Normalize
    feature_grads /= len(loader.dataset)
    return feature_grads.cpu().numpy()

# Store results in a dictionary
feature_importance_results = {}

# Analyze feature importance for all models
for model_name, model in models.items():
    print(f"\n=== Feature Importance for {model_name} ===")
    feature_scores = analyze_feature_importance(model, train_loader, device)
    
    # Store scores in dictionary
    feature_importance_results[model_name] = feature_scores

# Dynamically generate feature names based on actual feature count
num_features = len(next(iter(feature_importance_results.values())))
base_features = ["P (Active Power)", "Q (Reactive Power)", "X (Impedance)", "lr (Line Rating)"]
new_features = ["Edge BC", "Load %", "Elec BC"]
feature_names = base_features + new_features

# If there are unexpected additional features, append generic labels
if len(feature_names) < num_features:
    extra_features = [f"Extra Feature {i+1}" for i in range(num_features - len(feature_names))]
    feature_names += extra_features

# Format results in a readable table
df_importance = pd.DataFrame(feature_importance_results, index=feature_names[:num_features])
df_importance.index.name = "Feature"
df_importance.columns.name = "Model"

# Save to CSV
df_importance.to_csv("feature_importance_results.csv")
print("\nFeature Importance Results saved to 'feature_importance_results.csv'.")

# Print table
print(df_importance)



=== Feature Importance for GINE ===

=== Feature Importance for GAT ===

=== Feature Importance for GraphConv ===

Feature Importance Results saved to 'feature_importance_results.csv'.
Model                   GINE       GAT  GraphConv
Feature                                          
P (Active Power)    1.154666  2.117865   0.107767
Q (Reactive Power)  1.243408  0.473009   3.750725
X (Impedance)       1.149516  4.266479   1.806939
lr (Line Rating)    1.255445  3.786949   4.867756
Edge BC             1.847913  1.463134   3.827049
...                      ...       ...        ...
Extra Feature 60    0.174633  0.130924   0.055824
Extra Feature 61    0.303501  0.351021   0.151869
Extra Feature 62    0.108771  0.153942   0.086767
Extra Feature 63    0.720886  0.485142   0.404887
Extra Feature 64    0.524252  0.726933   0.405582

[71 rows x 3 columns]
